In [1]:
!nvidia-smi

Fri Aug 20 06:50:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-FHHL...  On   | 00000000:B1:00.0 Off |                    0 |
| N/A   35C    P0    25W / 150W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install fastai --upgrade

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 194kB 4.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 26.0MB/s eta 0:00:01
     |████████████████████████████████| 831.4MB 12.1MB/s eta 0:00:01    |███▌                            | 92.1MB 13.8MB/s eta 0:00:54     |█████▋                          | 146.3MB 9.2MB/s eta 0:01:15     |█████▉                          | 151.2MB 16.4MB/s eta 0:00:42     |██████                          | 153.1MB 16.4MB/s eta 0:00:42     |██████▉                         | 178.0MB 9.9MB/s eta 0:01:07     |███████                         | 179.1MB 9.9MB/s eta 0:01:07     |███████▏                        | 186.3MB 12.3MB/s eta 0:00:53     |███████▋                        | 199.0MB 13.6MB/s eta 0:00:47     |███████▊                        | 201.4MB 13.6MB/s eta 0:00:47     |███████████▍                    | 294.6MB 8.2MB/s eta 0:01:06     |████████████▍                   | 322.0MB 8.4MB/s eta 0:01:01

In [3]:
from fastai.text.all import *

1. Load data

In [5]:
path = untar_data(URLs.IMDB)

2. Fine-tune a language model on IMDb

In [6]:
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)

In [7]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos i do n't see the sense in going through so much trouble to make a movie like this , and then throw the history book out the window . xxmaj there was n't a single accurate detail in that movie other than the fact than xxmaj richtofen died , which i was grateful for at the end so i did n't have to watch any more . xxmaj movies like this","i do n't see the sense in going through so much trouble to make a movie like this , and then throw the history book out the window . xxmaj there was n't a single accurate detail in that movie other than the fact than xxmaj richtofen died , which i was grateful for at the end so i did n't have to watch any more . xxmaj movies like this are"
1,"xxmaj nikolaidis has the actors address the camera at various intervals . xxmaj the stand - out performance is that of the insane daughter , played by the extraordinary and outlandishly sexy xxmaj michele xxmaj valley . xxmaj panos xxmaj thanassoulis , as xxmaj sling , has virtually no dialog , but makes his mark in a pivotal but surprisingly neutral role . xxmaj black and white cinematography by xxmaj xxunk xxmaj","nikolaidis has the actors address the camera at various intervals . xxmaj the stand - out performance is that of the insane daughter , played by the extraordinary and outlandishly sexy xxmaj michele xxmaj valley . xxmaj panos xxmaj thanassoulis , as xxmaj sling , has virtually no dialog , but makes his mark in a pivotal but surprisingly neutral role . xxmaj black and white cinematography by xxmaj xxunk xxmaj xxunk"
2,"his girlfriend and dumping her body on the train . xxmaj mike , a loud little kid ( dale xxmaj belding ) , witnesses him . xxmaj harold tries to kill him , but is distracted and xxmaj mike runs away . \n\n xxmaj later , xxmaj harold ends up in a hotel / boarding house ( ? ) in a small town where he runs into xxmaj jean xxmaj maxwell (","girlfriend and dumping her body on the train . xxmaj mike , a loud little kid ( dale xxmaj belding ) , witnesses him . xxmaj harold tries to kill him , but is distracted and xxmaj mike runs away . \n\n xxmaj later , xxmaj harold ends up in a hotel / boarding house ( ? ) in a small town where he runs into xxmaj jean xxmaj maxwell ( the"
3,"playing with them . "" uncle "" finally gets a retrieve , when he develops a notion of parenting . xxmaj this calls attention , perhaps , to the "" dirty little secret "" not being confronted in our society about autism - that it really has a lot to do with neglecting children . xxmaj the details of xxmaj tian - bai 's upbringing that lead to his murderous behavior are","with them . "" uncle "" finally gets a retrieve , when he develops a notion of parenting . xxmaj this calls attention , perhaps , to the "" dirty little secret "" not being confronted in our society about autism - that it really has a lot to do with neglecting children . xxmaj the details of xxmaj tian - bai 's upbringing that lead to his murderous behavior are almost"
4,that boob from a gas station grease pit does not work at all - it just ruins the movie . i think in retrospect it 's not the whole movie i hate so much as the fact that ' the boob ' is so obviously not someone who would be on a space ship - not even to ' grease the hatch . ' xxup omg . i wonder if he checked,boob from a gas station grease pit does not work at all - it just ruins the movie . i think in retrospect it 's not the whole movie i hate so much as the fact that ' the boob ' is so obviously not someone who would be on a space ship - not even to ' grease the hatch . ' xxup omg . i wonder if he checked the


In [10]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [11]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.425848,4.118347,0.285327,61.457550,16:54


In [13]:
learn.save('1epoch')
learn = learn.load('1epoch')

In [14]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.074031,3.915394,0.304144,50.168850,17:34
1,4.014558,3.869981,0.308912,47.941479,17:11
2,3.966302,3.811498,0.315168,45.218143,17:06
3,3.918932,3.762368,0.320792,43.050266,17:02
4,3.859914,3.746400,0.322813,42.368267,17:06


In [15]:
learn.save_encoder('finetuned')

3. Fine-tune a classifier

In [16]:
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', text_vocab=dls_lm.vocab)

In [17]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [18]:
learn = learn.load_encoder('finetuned')

In [19]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.242223,0.191658,0.924960,01:49


In [20]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.218153,0.172692,0.933640,02:38


In [21]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.189482,0.159275,0.940680,03:52


In [22]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.174046,0.154317,0.942520,04:36
1,0.134750,0.151739,0.943800,04:46
